In [25]:
import json
from pandas import DataFrame
import matplotlib.pyplot as plt
import pandas as pd
from score import compute_score, compute_score_from_weighted_incident_rate
from bokeh.plotting import figure, output_file, show
from bokeh.resources import INLINE
import bokeh.io
# bokeh.io.output_notebook(INLINE)
from bokeh.io import output_file, show
# from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.layouts import row

In [26]:
mylist = []
for line in open('driver-16703.json'):
    mylist.append(json.loads(line.rstrip()))

# [json.loads(line.rstrip() for line in open('filename')] this is equivalent to the code above-written as a comprehension
#pandas.read_json

df = DataFrame.from_records(mylist)
df =  df.fillna(0)

df_sorted = df.sort_values('id')
data_cst = df_sorted
# print(df_sorted.info())
# print(df_sorted.head())
# print(df_sorted['id'])

#Need to put code here to select just one month of data at a time

data_cst = data_cst.groupby(data_cst['id'], axis=0).sum()
data_cst = data_cst.reset_index()
print(data_cst.head())
print(data_cst.info())

       id    c0k    c0m  c10k  c10m  c11k  c11m  c12k  c12m  c13k  ...    \
0  471053  153.0  153.0  13.0   0.0   2.0   0.0   0.0   0.0   0.0  ...     
1  471377  222.0  222.0  27.0   1.0   0.0   0.0   0.0   0.0   0.0  ...     
2  471381  165.0  165.0  18.0   0.0   0.0   0.0   0.0   0.0   1.0  ...     
3  471383  202.0  202.0  22.0   0.0   1.0   0.0   0.0   0.0   0.0  ...     
4  471385  175.0  175.0  12.0   0.0   0.0   0.0   0.0   0.0   0.0  ...     

    u5m  u6k   u6m   u7k  u7m   u8k  u8m  u9k  u9m   uobs  
0  16.0  0.0  29.0  13.0  1.0  15.0  0.0  1.0  1.0  463.0  
1  44.0  0.0  80.0  11.0  1.0  44.0  0.0  0.0  0.0  699.0  
2  50.0  2.0  33.0  19.0  4.0  50.0  3.0  3.0  0.0  458.0  
3  63.0  2.0  69.0  38.0  2.0  62.0  5.0  3.0  2.0  676.0  
4  21.0  0.0  27.0  13.0  0.0  21.0  0.0  0.0  0.0  548.0  

[5 rows x 95 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 95 columns):
id       529 non-null int64
c0k      529 non-null float

In [27]:
def get_total_counts(item):
    count0_5m = (item['c1m'] + item['c2m'] + item['c3m'] + item['c4m'] + item['c5m'] +
                 item['u1m'] + item['u2m'] + item['u3m'] + item['u4m'] + item['u5m'])

    count6_10m = (item['c6m'] + item['c7m'] + item['c8m'] + item['c9m'] + item['c10m'] +
                  item['u6m'] + item['u7m'] + item['u8m'] + item['u9m'] +  item['u10m'])

    count11_15m = (item['c11m'] + item['c12m'] + item['c13m'] + item['c14m'] + item['c15m'] +
                   item['u11m'] + item['u12m'] + item['u13m'] + item['u14m'] + item['u15m'])

    count16_upm = item['c16m+'] + item['u16m+']
    return count0_5m, count6_10m, count11_15m, count16_upm

ct_observations = data_cst['obs']
data_cst['count_total_observations'] = ct_observations
(data_cst['count0_5M'],
data_cst['count6_10M'],
data_cst['count11_15M'],
data_cst['count16_upM']) = get_total_counts(data_cst)

In [28]:
def compute_weighted_values(item):

    item['obs'] = item['obs'].astype(int)
    item['count0_5M'] = item['count0_5M'].astype(int)
    item['count6_10M'] = item['count6_10M'].astype(int)
    item['count11_15M'] = item['count11_15M'].astype(int)
    item['count16_upM'] = item['count16_upM'].astype(int)

    i = item['count0_5M'] + item['count6_10M'] + item['count11_15M'] + item['count16_upM']
    weighted_i = (item['count0_5M'] + (item['count6_10M'] * 4) + (item['count11_15M'] * 7) + (item['count16_upM']*10))

    weighted_obs = (item['obs'] - i + weighted_i)
    weighted_i_rate= round(weighted_i.astype(float) / weighted_obs.astype(float) * 100.0, 2)
    return i, weighted_i, weighted_obs, weighted_i_rate

(data_cst['i'],
data_cst['weighted_i'],
data_cst['weighted_obs'],
data_cst['weighted_i_rate']) = compute_weighted_values(data_cst)

In [29]:
SCALE_MIN = 350
SCALE_MAX = 850

scale_range = SCALE_MAX - SCALE_MIN
data_cst['score'] = round((100 - data_cst['weighted_i_rate']) * scale_range / 100.0 + SCALE_MIN, 2)

# print(data_cst.info())
# print(data_cst.head())

data_cst['id'] = data_cst['id'].astype(str)

score_sorted = data_cst.sort_values('score')
score_sorted = score_sorted.set_index('id')
score_sorted = score_sorted.reset_index()
# print(score_sorted.info())
# print(score_sorted.head())

In [35]:
drivers_scores = score_sorted[['id', 'score', 'count_total_observations', 'count0_5M', 'count6_10M', 'count11_15M', 'count16_upM']]
drivers_scores['total_incidents'] = (drivers_scores['count0_5M'] + drivers_scores['count6_10M'] + drivers_scores['count11_15M'] + drivers_scores['count16_upM'])
drivers_scores['pct_incidents'] = (drivers_scores['total_incidents'] / drivers_scores['count_total_observations']) * 100							


drivers_scores['pct0_5M'] = (drivers_scores['count0_5M'] / drivers_scores['count_total_observations']) * 100
drivers_scores['pct6_10M'] = (drivers_scores['count6_10M'] / drivers_scores['count_total_observations']) * 100
drivers_scores['pct11_15M'] = (drivers_scores['count11_15M'] / drivers_scores['count_total_observations']) * 100
drivers_scores['pct16_upM'] = (drivers_scores['count16_upM'] / drivers_scores['count_total_observations']) * 100

# print(drivers_scores.head())
# print(drivers_scores.tail())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [33]:
output_file("pct_incidents_16703.html")
source= ColumnDataSource(drivers_scores)
hover = HoverTool(tooltips=[('id', '@id'), ('pct0_5M', '@pct0_5M'), ('pct6_10M', '@pct6_10M'), ('pct11_15M', '@pct11_15M'), ('pct16_upM', '@pct16_upM')])
p1 = figure(title="Total Observations vs. % Incidents", title_location="above", y_range=(-5, 80), plot_width=600, plot_height=400, tools=[hover, 'pan'])
p1.xaxis[0].axis_label = 'Total Observations'
p1.yaxis[0].axis_label = '% Incidents'
p1.circle(x = 'count_total_observations', color='red', y= 'pct_incidents', size=7, source=source)
p2 = figure(title="Score vs. % Incidents", title_location="above", y_range = p1.y_range, plot_width=600, plot_height=400, tools=[hover, 'pan'])
p2.xaxis[0].axis_label = 'Score'
p2.yaxis[0].axis_label = '% Incidents'
p2.circle(x = 'score', y= 'pct_incidents', color='red', size=7, source=source)
p2.add_tools(hover)
show(row(p1, p2))


In [34]:
output_file("pct_ranges_score_16703.html")
source= ColumnDataSource(drivers_scores)
hover = HoverTool(tooltips=[('id', '@id')])
p = figure(title="Score vs. % Incidents", title_location="above", y_range = (-5, 60), plot_width=600, plot_height=400, tools=[hover, 'pan'])
p.xaxis[0].axis_label = 'Score'
p.yaxis[0].axis_label = '% Incidents'
p.circle(x = 'score', y= 'pct0_5M', size=7, source=source, legend='% 0-5M')

p.circle(x = 'score', y = 'pct6_10M', size=7, color='red', source=source, legend='% 5-10M')
p.circle(x = 'score', y = 'pct11_15M', size=7, color='orange', source=source, legend='% 11-15M')
p.circle(x = 'score', y = 'pct16_upM', size=7, color='green', source=source, legend='% 16+M')
p.add_tools(hover)
show(p)